In [ ]:
# !pip install pyaudio
import speech_recognition as sr
import tensorflow as tf 
import tensorflow_hub as hub
import pyttsx3
import torch
import tensorflow as tf
import torch.nn as nn
import os
import numpy as np

In [ ]:
###BASELINE

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ITSM
input_size = 512
hidden_size = 220
num_classes = 11
num_epochs = 20
batch_size = 40
learning_rate = 0.0005

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [ ]:


path = "Models/acc_0.9001107498887677"
dic_path = path + ".pickle"

input_model = NeuralNet(input_size, hidden_size, num_classes).to(device)
input_model.load_state_dict(torch.load(path))
# input_model = model

with open(dic_path, 'rb') as handle:
    dic = pickle.load(handle)
    
rev = {}
for k in list(dic.keys()):
    v = dic[k]
    rev[v] = k

rec = sr.Recognizer()
with sr.Microphone() as source:
    print("I am the mighty Jarvis!" + '\n' + "Ask me something")
    audio = rec.listen(source)
    print("Thinking...")
try:
    sample = rec.recognize_google(audio)
except:
    sample = ""

if "*" in sample:
    heard = "HEY!"
    message = "Watch your language!"
elif sample != "":

    text = [sample]
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    vectors = np.array(embed(text)).tolist()
    vectors = torch.tensor(vectors)


    with torch.no_grad():
        outputs = input_model(vectors.to(device))
        maximum, res1 = torch.max(outputs.data, 1)
    res1 = res1.tolist()
    message = "The intent for your request is: " + rev[res1[0]]
    heard = "I heard: " + sample
    if maximum.item() < 1.0:
        heard = "Sorry, this question is not applicable"
        message = "Please try again"
    
else:
    heard = "Sorry, I could not hear that."
    message = "Please try again"

engine = pyttsx3.init()
engine.say(heard)
print(heard)
engine.say(message)
print(message)
engine.runAndWait()




In [ ]:
from gtts import gTTS 
msg = "I am the mighty Jarvis!" + '\n' + "Ask me something"
gTTS(text=msg, lang='en', slow=False) 

  